In [41]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import random 

In [20]:
# Carrega o data set
data = load_iris()

# Atribui valores à entrada e à saída 
X = data.data
y = data.target

In [26]:
# Separando dados para treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Dados para treinamento: {len(X_train)}\nDados para teste: {len(X_test)}')

Dados para treinamento: 120
Dados para teste: 30


### PARÂMETROS GLOBAIS

In [64]:
CLASS_NUM = 3
POPULATION_SIZE = 60
SELECTION_RATE = 0.2 # 20%


### GERAÇÃO DA POPULAÇÃO DE ANTICORPOS

In [28]:
# Classe Anticorpo

class Antibody:
    def __init__(self, vector, label):
        self.vector = vector
        self.label = label
        self.affinity = None

In [55]:
# Classe Antígeno

class Antigen:
    def __init__(self, vector, label):
        self.vector = vector
        self.label = label

Para a geração da população, iremos escolher aleatoriamente os valores do vetor, dentro de um limite estimado pelo menor e maior valor de cada coluna do dataset.

Por fim, iremos atribuir igualmente as labels para cada anticorpo,de modo que cada classe tenha a mesma quantidade de anticorpos gerados.

In [40]:
# Extraindo o valor minimo e máximo de cada atributo da Iris

min_sepal_length = min(X_train[:, 0])
max_sepal_length = max(X_train[:, 0])

min_sepal_width = min(X_train[:, 1])
max_sepal_width = max(X_train[:, 1])

min_petal_length = min(X_train[:, 2])
max_petal_length = max(X_train[:, 2])

min_petal_width = min(X_train[:, 3])
max_petal_width = max(X_train[:, 3])

print(f'Limite do comprimento da sépala: [{min_sepal_length}, {max_sepal_length}]\nLimite da largura da sépala: [{min_sepal_width}, {max_sepal_width}]\nLimite do comprimento da pétala: [{min_petal_length}, {max_petal_length}]\nLimite da largura da sépala: [{min_petal_width}, {max_petal_width}]\n')

Limite do comprimento da sépala: [4.3, 7.7]
Limite da largura da sépala: [2.0, 4.4]
Limite do comprimento da pétala: [1.0, 6.7]
Limite da largura da sépala: [0.1, 2.5]



In [ ]:
# Função para gerar um anticorpo

def generateAntibody(label):

    # Escolhe um valor aleatorio dentro da faixa estimada
    vector = [
        random.uniform(min_sepal_length, max_sepal_length),
        random.uniform(min_sepal_width, max_sepal_width),
        random.uniform(min_petal_length, max_sepal_length),
        random.uniform(min_petal_width, max_petal_width)
    ]
    
    antibody = Antibody(vector, label)
    return antibody

In [ ]:
# Função para criar a população com base no seu tamanho e numero de classes

def generatePopulation():

    population = []
    # Cria labels igualmente com base no numero de classes
    labels = [i % CLASS_NUM for i in range(POPULATION_SIZE)]

    for i in range(POPULATION_SIZE):
        population.append(generateAntibody(labels[i]))
    
    return population

### Avaliação do Fitness

In [53]:
# Função para a avaliação do fitness em relação ao antígeno
# O maior fitness possível será igual a 1

def evaluateFitness(antibody, antigen):
    # Caso possuam a mesma label
    if antibody.label == antigen.label:
    
        """
        Utiliza-se o calcúlo do inverso da distância Euclidiana
        para avaliar o fitness. Logo

        Nota = 1/Distancia(anticorpo, antigeno)

        quanto menor a distancia, maior será  nota
        """
        # Método do NumPy que calcula a distância euclidiana entre dois pontos
        distance = np.linalg.norm(np.array(antibody.vector) - np.array(antigen.vector))

        return 1/(1 + distance) # Operação para evitar distância com zero
    # Significa que o anticorpo eh um falso positivo   
    else: 
        return 0


In [ ]:
# Célula para realização de teste
pop = generatePopulation() # Gera uma população
antigen = Antigen(X_train[0], y_train[0]) # Pega o primeiro antigeno

# Realiza a avaliação de cada anticorpo da população
print(f'Para o Antígeno {antigen.label} de vetor {antigen.vector}\n')
for antibody in pop:
    print(f'Anticorpo {antibody.label}, {antibody.vector} : {evaluateFitness(antibody, antigen)}\n')

Para o Antígeno 0 de vetor [4.6 3.6 1.  0.2]

Anticorpo 0, [6.316908703315413, 3.7964986294338043, 7.551554524057862, 2.4516491130990383] : 0.12285056925792401

Anticorpo 1, [7.492276261585225, 4.374475228768263, 3.044885188796934, 1.7115298274150743] : 0

Anticorpo 2, [6.821563570381018, 2.2974642178329003, 7.148431035348683, 1.3584063618688864] : 0

Anticorpo 0, [7.419995622398515, 4.240986131491162, 1.839821713466541, 1.0007543546545874] : 0.24295157240965035

Anticorpo 1, [7.213680749409002, 3.024174529297866, 6.872729807436186, 1.9620084114244474] : 0

Anticorpo 2, [7.562002716246085, 2.0243341779333592, 4.52065831086544, 2.036720452682238] : 0

Anticorpo 0, [7.047287379164613, 2.7040800166433634, 6.777651463568613, 1.995869549527432] : 0.13179158261716153

Anticorpo 1, [6.424498700101229, 3.321091396861932, 1.8666293433353176, 1.232744909657713] : 0

Anticorpo 2, [7.4735154979291, 2.278318243129049, 6.048433587013066, 0.2814616787839981] : 0

Anticorpo 0, [6.727948377386975, 4.17

### Seleção dos N melhores anticorpos

In [ ]:
# Função para realizar a seleção

def selectBestFitness(evaluated_antibodies):
    # Ordena a lista do menor para o maior fitness
    ordered_list = sorted(evaluated_antibodies, key=lambda x : x[0], reverse=True)

    # Determina a quantidade de melhores com base no SELECTION RATE
    best_qtd = int(POPULATION_SIZE * SELECTION_RATE)
    # Seleciona essa quantidade na lista
    bests = ordered_list[:best_qtd]
    return bests # Retorna a parcela da lista selectionada

In [ ]:
# Célula para realização de teste

# Lista com os anticorpos e seus respectivos fitness
evaluated_antibodies = []

for antibody in pop:
    evaluated_antibody = []
    # Avalia o fitness de cada anticorpo
    evaluated_antibody.append(evaluateFitness(antibody, antigen))
    evaluated_antibody.append(antibody)
    # Adiciona a lista com o anticorpo e seu fitness
    evaluated_antibodies.append(evaluated_antibody)

# Resgata os N melhores fitness
bests = selectBestFitness(evaluated_antibodies)

#Imprime
for i ,antibody in enumerate(bests):
    print(f'Anticorpo {i + 1} : {antibody[0]}')

Anticorpo 1 : 0.2792728829808624
Anticorpo 2 : 0.26896030504769014
Anticorpo 3 : 0.2559657755250283
Anticorpo 4 : 0.2529529169426308
Anticorpo 5 : 0.24295157240965035
Anticorpo 6 : 0.24209867171854024
Anticorpo 7 : 0.24189623504913946
Anticorpo 8 : 0.2287081763275899
Anticorpo 9 : 0.2281367733457791
Anticorpo 10 : 0.21559107171692124
Anticorpo 11 : 0.19855642896896775
Anticorpo 12 : 0.18231834884657963


### Clonagem dos anticorpos mais promissores

Para a etapa de clonagem, utilizamos o seguinte equação para definir quantos clones cada fitness irá receber:

$$
QC^i_k = \left( \frac{af^i_k}{\sum_{k=1}^{n} af^i_k} \right) \cdot Cl
$$

Onde:
- `QC^i_k`: Quantidade de clones gerados para o anticorpo *k* na iteração *i*
- `af^i_k`: Afinidade do anticorpo *k* na iteração *i*
- `Cl`: Quantidade total de clones a serem distribuídos
- `n`: Número total de anticorpos selecionados

In [74]:
# Função para realizar a clonagem

def clone(bests):

    # Lista final para armazenar os clone
    clones = []

    # Define o numero total de clone
    total_clones = POPULATION_SIZE - len(bests)

    # Somatorio de afinidades
    fitness = [x[0] for x in bests] # cria lista de afinidades
    fitness_sum = sum(fitness) # Soma total das afinidades

    # Lista temporária para armazenar as classes de anticorpos
    antibodies = [x[1] for x in bests] # cria uma lista de anticorpos

    # Calcula a quantidade de clones para cada fitness e armazena em uma lista
    qtd_clones = [round((c/fitness_sum) * total_clones) for c in fitness]

    # Realiza a clonagem ideal para cada fitness e armazena na lista final de clones
    counter = 0
    while counter < len(qtd_clones):
        for i in range(qtd_clones[counter]):
            # Lista que armazena a classe do anticorpo e seu fitness
            pair = []
            pair.append(fitness[counter])
            pair.append(antibodies[counter])
            # Salva na lista final
            clones.append(pair)
        counter += 1
    
    # Retorna o total de clones
    return clones

In [75]:
# Célula para a realização de teste
clones = clone(bests)
counter = 0

for i, clone in enumerate(clones):
    print(f'Clone {i + 1}: {clone[0]}')
    counter+= 1
    

print(f'Total de clones gerados: {counter}')

Clone 1: 0.2792728829808624
Clone 2: 0.2792728829808624
Clone 3: 0.2792728829808624
Clone 4: 0.2792728829808624
Clone 5: 0.2792728829808624
Clone 6: 0.26896030504769014
Clone 7: 0.26896030504769014
Clone 8: 0.26896030504769014
Clone 9: 0.26896030504769014
Clone 10: 0.26896030504769014
Clone 11: 0.2559657755250283
Clone 12: 0.2559657755250283
Clone 13: 0.2559657755250283
Clone 14: 0.2559657755250283
Clone 15: 0.2529529169426308
Clone 16: 0.2529529169426308
Clone 17: 0.2529529169426308
Clone 18: 0.2529529169426308
Clone 19: 0.24295157240965035
Clone 20: 0.24295157240965035
Clone 21: 0.24295157240965035
Clone 22: 0.24295157240965035
Clone 23: 0.24209867171854024
Clone 24: 0.24209867171854024
Clone 25: 0.24209867171854024
Clone 26: 0.24209867171854024
Clone 27: 0.24189623504913946
Clone 28: 0.24189623504913946
Clone 29: 0.24189623504913946
Clone 30: 0.24189623504913946
Clone 31: 0.2287081763275899
Clone 32: 0.2287081763275899
Clone 33: 0.2287081763275899
Clone 34: 0.2287081763275899
Clone 

### Hipermutação dos anticorpos de acordo com a afinidade